In [1]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [2]:
BASE = "https://www.courts.state.hi.us"
start = "https://www.courts.state.hi.us/self-help/courts/forms/court_forms"

soup = connect_to_page(start)

cats = soup.find_all("a", href = True)
groups_url = []
groups_names = []

for i in range(len(cats)):
    if ("forms" in cats[i]['href']) and "first" not in cats[i]['href'] and "second" not in cats[i]['href'] and  "third" not in cats[i]['href'] and "fifth" not in cats[i]['href']:
        if "http" not in cats[i]['href']:
            groups_url.append(BASE+cats[i]['href'])
        else:
            groups_url.append(cats[i]['href']) 
        groups_names.append(cats[i].text)

groups_url = groups_url[5:len(groups_url)-4] #first 5 and last 4 useless
groups_names = groups_names[5:len(groups_names)-4]


In [ ]:
pattern = r"\([^()]*\)"
pdfs = {}
form_dest = "All_forms/"
jur = 'HI'
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","error"])
today = date.today().strftime("%Y-%m-%d")


for i in range(len(groups_url)):
    soup = connect_to_page(groups_url[i])
    group = groups_names[i]
    print(group,"----------------------------")
    for link in soup.find_all("a", href = True):
        if "pdf" in link['href']:
            if "http" in link['href']:
                url = link['href']
            else:
                url = BASE+link['href']
            # print(url)

            if "1F-P-" in link.text or "1F-E-" in link.text:
                filename = link.text
                prevtd = link.findParent("tr")
                # prevtr = prevtd.findParent()
                # td = prevtr.find("a")
                title = re.sub(pattern,"",prevtd.find("td").text)
                print(title)
                print(filename)
                print(url)
                print("--------TOP-------------")
            elif "KMH-" in link.text:
                filename = link.text.split(":")[0]
                filename = filename[:len(filename)]
                title = re.sub(pattern,"",link.text.split(":")[1])
                print(title)
                print(filename+".pdf")
                print("--------TOP-BOTTOM-------------")
            elif "PDF" in link.text:
                filename = url.split("/")[-1]
                prevtr = link.findPrevious("tr")
                td = prevtr.find_all("td")[1]
                title = re.sub(pattern, "",td.find("p").text)
                print(title)
                print(filename)
                print("--------BOTTOM-------------")
            else:
                title = re.sub(pattern,"",link.text)
                # nexttd = link.findNextSibling("td")
                filename = url.split("/")[-1]
                if filename=="1FP818.pdf":
                    continue
                else:
                    print(title)
                print(filename)
                print(url)
                print("--------BOTTOM-BOTTOM-------------")
            if filename not in pdfs:
                pdfs[filename] = title
                id = hashme(url)
                status = download_pdf(url,form_dest,id+".pdf")
                error = 0
                if status=="error":
                    print("###########################")
                    print("ERROR", url, group)
                    print("###########################")
                    error = 1
                files_df = files_df.append(pd.DataFrame([[id,jur,start,title,group,url,filename,today,error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "error"]))

files_df.to_csv("HI_metaData.csv",index=False, encoding="utf-8")

In [ ]:
import pandas as pd
df = pd.read_csv ('HI_metaData.csv')
df_set = set()

for i in df["id"]:
    if i not in df_set:
        df_set.add(i)
    else:
        print(i)

